In [2]:
from pythainlp.tag.thainer import ThaiNameTagger
from pythainlp.tokenize import word_tokenize
from pythainlp.util import normalize
from pythainlp.tag import chunk_parse
from pythainlp.tag import pos_tag
from pythainlp.tag import tag_provinces
import nltk
import spacy
import sqlite3
import os
import csv
import json
import re
cur_dir = globals()['_dh'][0]

class locationExtractor(object):
    def __init__(self, news_body, db_file=None):
        # need to use __file__ in script
        db_file = db_file or os.path.dirname(os.path.realpath(cur_dir)) + "/locationdata.db"
        self.conn = sqlite3.connect(db_file)
        self.news_body = news_body
        
    def populate_db(self):
        cur = self.conn.cursor()
        
        cur.execute("CREATE TABLE IF NOT EXISTS locations(AD_LEVEL,TA_ID,TAMBON_T,TAMBON_E,AM_ID,AMPHOE_T,AMPHOE_E,CH_ID,CHANGWAT_T,CHANGWAT_E,LAT,LONG)")
        
        with open(str(cur_dir)+"/data/tambon.csv", encoding = 'UTF') as info:
            reader = csv.reader(info)
            for row in reader:
                cur.execute("INSERT INTO locations VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);", row)
            
            self.conn.commit()
        
    def db_has_data(self):
        cur = self.conn.cursor()
        
        cur.execute("SELECT Count(*) FROM sqlite_master WHERE name='locations';")
        data = cur.fetchone()[0]
        
        if data > 0:
            cur.execute("SELECT Count(*) FROM locations")
            data = cur.fetchone()[0]
            return data > 0
        
        return False
    
    def get_ner(self):
        if len(self.news_body) > 0:
            ner = ThaiNameTagger()
            nes = ner.get_ner(self.news_body)
            l = []
            # print(nes)
            for ne in nes:
                if "LOCATION" in ne[2] or "PERSON" in ne[2] or "ORGANIZATION" in ne[2] or "DATE" in ne[2] or " " in ne[0] or "O" in ne[2]:
                    l.append(ne[0])
                    
                    
            #print('list')
            #print(l)
            text  = "".join(l)
            text = re.sub(' +', ' ', text)
            # print(text)
            location = text.split()
            
                    
                      
                    
           
            return location
            
    
    def find_province(self):
        
        self.province = ''
        
        words = word_tokenize(self.news_body)
        print(words)
        wordTags = tag_provinces(words)
        for word in wordTags:
            if "LOCATION" in word[1]:
                self.province = word[0]
        if not self.province:
            if not self.db_has_data():
                self.populate_db()
            cur = self.conn.cursor()
            # location = 'บางบอนนะๆๆ'
            
            # query = "SELECT * FROM locations WHERE AMPHOE_T LIKE '%" + location + "%'"
            provinceQuery  = "SELECT CHANGWAT_T FROM locations"
            
            
            
            
         
    def find_tambon(self):

        self.tambon = []
        location = self.get_ner()
        print(location)
        for i in range(len(location)):
            if 'ต.' in location[i] or 'ตำบล' in location[i] or 'แขวง' in location[i]:
                self.tambon.append(location[i])
         
        if not self.db_has_data():
                self.populate_db()
                
        cur = self.conn.cursor()

        tambonQuery  = "SELECT TAMBON_T FROM locations"
        cur.execute("SELECT TAMBON_T FROM locations")
        rows = cur.fetchall()
        print(rows)
        
        
    def find_amphoe(self):
        
        self.amphoe = []
        location = self.get_ner()
        for i in range(len(location)):
            if 'อ.' in location[i] or 'อำเภอ' in location[i] or 'เขต' in location[i]:
                self.amphoe.append(location[i])
        
        
        
 
    def find_latlong(self):
        pass
    
    
    def clean_text(self, text):
        if "ต." in text:
            text = text.replace("ต.","")
        elif "อ." in text:
            text = text.replace("อ.","")
        elif "จ." in text:
            print('hello')
            text = text.replace("จ.","")
        else:
            return text
        return text
    
    def find_location(self):
        location = {"amphoe": self.amphoe,
                    "tambon": self.tambon,
                    "province": self.province
                   }
        
        
        
    
    def info_validate(self):
        if not self.db_has_data():
            self.populate_db()
        
        
        
         
                
                
                    
                
    
    
        

In [3]:
test = locationExtractor(news_body='เมื่อเวลา 15.00 น. วันที่ 16 สิงหาคม 2565 พ.ต.อ.จิรพัฒน์ พรหมสิทธิการ ผกก.สน.เตาปูน พร้อมด้วย พ.ต.ท.สุรเดช ฉัตรไทย รอง ผกก.สส.สน.เตาปูน พ.ต.ท.ไพรวัลย์ ธนารมย์ สว.สส.สน.เตาปูน พร้อมเจ้าหน้าที่ตำรวจฝ่ายสืบสวน สน.เตาปูน ร่วมกันจับกุมตัว นายประเทือง ขุ่ยทอง อายุ 51 ปี ผู้ต้องหาคดีลักทรัพย์ จับกุมได้ที่ ห้องเช่าเลขที่ 122 ซอยสมเด็จพระเจ้าตากสิน 4 แขวงบางยี่เรือ เขตธนบุรี กทม.')
test.find_amphoe()
test.amphoe
# เอา ตัว database มากรองจาก list อีกทีนึง น่อวว  

['พ.ต.อ.จิรพัฒน์', 'เขตธนบุรี']

In [4]:
test.find_tambon()
test.tambon 

['เมื่อเวลา', 'วันที่', '16', 'สิงหาคม', '2565', 'พ.ต.อ.จิรพัฒน์', 'พรหมสิทธิการ', 'ผกก.สน.เตาปูน', 'พร้อมด้วย', 'พ.ต.ท.สุรเดช', 'ฉัตรไทย', 'รอง', 'ผกก.สส.สน.เตาปูน', 'พ.ต.ท.ไพรวัลย์', 'ธนารมย์', 'สว.สส.สน.เตาปูน', 'พร้อมเจ้าหน้าที่ตำรวจฝ่ายสืบสวน', 'สน.เตาปูน', 'ร่วมกันจับกุมตัว', 'นายประเทือง', 'ขุ่ยทอง', 'อายุ', 'ผู้ต้องหาคดีลักทรัพย์', 'จับกุมได้ที่', 'ห้องเช่าเลขที่', '122', 'ซอยสมเด็จพระเจ้าตากสิน', '4', 'แขวงบางยี่เรือ', 'เขตธนบุรี', 'กทม.']
[('TAMBON_T',), ('ต. เกาะสาหร่าย',), ('ต. มาบตาพุด',), ('ต. มาบตาพุด',), ('ต. มาบตาพุด',), ('ต. มาบตาพุด',), ('ต. หาดทรายรี',), ('ต. ด่านสวี',), ('ต. ท่าตูม',), ('ต. อ่างทอง',), ('ต. อ่างทอง',), ('ต. อ่างทอง',), ('ต. สัตหีบ',), ('ต. ตะเสะ',), ('ต. หาดทรายรี',), ('ต. ปากคลอง',), ('ต. ปากตะโก',), ('ต. ปากคลอง',), ('ต. เกาะสาหร่าย',), ('ต. อ่างทอง',), ('ต. หาดทรายรี',), ('ต. ปากตะโก',), ('ต. เกาะสาหร่าย',), ('ต. ตะเสะ',), ('ต. ด่านสวี',), ('ต. ปากตะโก',), ('ต. ตะเสะ',), ('ต. ปากคลอง',), ('ต. บางกะไชย',), ('ต. ตะเสะ',), ('ต. ปากคลอง',), ('ต. เกา

['พ.ต.อ.จิรพัฒน์', 'พ.ต.ท.สุรเดช', 'พ.ต.ท.ไพรวัลย์', 'แขวงบางยี่เรือ']

In [5]:
test.find_province()
test.province

['เมื่อ', 'เวลา', ' ', '15.00', ' ', 'น.', ' ', 'วันที่', ' ', '16', ' ', 'สิงหาคม', ' ', '2565', ' ', 'พ.ต.อ.', 'จิร', 'พัฒน', '์', ' ', 'พรหม', 'สิทธิ', 'การ', ' ', 'ผกก.', 'สน.', 'เตา', 'ปูน', ' ', 'พร้อมด้วย', ' ', 'พ.ต.ท.', 'สุร', 'เดช', ' ', 'ฉัตร', 'ไทย', ' ', 'รอง', ' ', 'ผกก.', 'สส.', 'สน.', 'เตา', 'ปูน', ' ', 'พ.ต.ท.', 'ไพร', 'วัลย์', ' ', 'ธนา', 'รมย์', ' ', 'สว', '.', 'สส.', 'สน.', 'เตา', 'ปูน', ' ', 'พร้อม', 'เจ้าหน้าที่ตำรวจ', 'ฝ่าย', 'สืบสวน', ' ', 'สน.', 'เตา', 'ปูน', ' ', 'ร่วมกัน', 'จับกุม', 'ตัว', ' ', 'นาย', 'ประเทือง', ' ', 'ขุ่ย', 'ทอง', ' ', 'อายุ', ' ', '51', ' ', 'ปี', ' ', 'ผู้ต้องหา', 'คดี', 'ลักทรัพย์', ' ', 'จับกุม', 'ได้ที่', ' ', 'ห้องเช่า', 'เลขที่', ' ', '122', ' ', 'ซอย', 'สมเด็จพระเจ้าตากสิน', ' ', '4', ' ', 'แขวง', 'บาง', 'ยี่', 'เรือ', ' ', 'เขต', 'ธนบุรี', ' ', 'กทม.']


''

In [6]:
# อันนี้เอาเข้าละผิด
text = "เมื่อเวลา 07.30 น.วันที่ 8 สิงหาคม 2565 ร.ต.อ.เอกอมร สีสัน รองสว.สอบสวนสภ.คูคต รับแจ้งมีเหตุยิงกันตายที่เกิดเหตุภายในตลาดสี่มุมเมืองอาคารเลขที่ 355/205-6 ซึ่งเปิดเป็นกิจการจำหน่ายพลาสติกชื่อร้านเจ๊ณีพลาสติก ม.15 ต.คูคต อ.ลำลูกกา จ.ปทุมธานี หลังรับแจ้งจึงรายงานให้ผู้บังคับบัญชาทราบพร้อมทั้งรีบรุดไปตรวจสอบที่เกิดเหตุ พร้อมด้วย พ.ต.อ.มนัสเวท ทองอิ่ม ผกก.สภ.คูคต เจ้าหน้าที่ชุดสืบสวน สภ.คูคต แพทย์ โรงพยาบาลภูมิพลอดุลยเดช เจ้าหน้าที่พิสูจน์หลักฐาน และอาสาสมัครมูลนิธิร่วมกตัญญู"
word_tokenize(text)

['เมื่อ',
 'เวลา',
 ' ',
 '07.30',
 ' ',
 'น.',
 'วันที่',
 ' ',
 '8',
 ' ',
 'สิงหาคม',
 ' ',
 '2565',
 ' ',
 'ร.ต.อ.',
 'เอก',
 'อมร',
 ' ',
 'สีสัน',
 ' ',
 'รอง',
 'สว',
 '.',
 'สอบสวน',
 'สภ.',
 'คูคต',
 ' ',
 'รับ',
 'แจ้ง',
 'มี',
 'เหตุ',
 'ยิง',
 'กัน',
 'ตาย',
 'ที่เกิดเหตุ',
 'ภายใน',
 'ตลาด',
 'สี่',
 'มุม',
 'เมือง',
 'อาคาร',
 'เลขที่',
 ' ',
 '355',
 '/',
 '205',
 '-',
 '6',
 ' ',
 'ซึ่ง',
 'เปิด',
 'เป็น',
 'กิจการ',
 'จำหน่าย',
 'พลาสติก',
 'ชื่อ',
 'ร้าน',
 'เจ๊',
 'ณี',
 'พลาสติก',
 ' ',
 'ม.',
 '15',
 ' ',
 'ต.',
 'คูคต',
 ' ',
 'อ.',
 'ลำ',
 'ลูก',
 'กา',
 ' ',
 'จ.',
 'ปทุมธานี',
 ' ',
 'หลัง',
 'รับ',
 'แจ้ง',
 'จึง',
 'รายงาน',
 'ให้',
 'ผู้บังคับบัญชา',
 'ทราบ',
 'พร้อมทั้ง',
 'รีบรุด',
 'ไป',
 'ตรวจสอบ',
 'ที่เกิดเหตุ',
 ' ',
 'พร้อมด้วย',
 ' ',
 'พ.ต.อ.',
 'มนัส',
 'เวท',
 ' ',
 'ทอง',
 'อิ่ม',
 ' ',
 'ผกก.',
 'สภ.',
 'คูคต',
 ' ',
 'เจ้าหน้าที่',
 'ชุด',
 'สืบสวน',
 ' ',
 'สภ.',
 'คูคต',
 ' ',
 'แพทย์',
 ' ',
 'โรงพยาบาล',
 'ภูมิพล',
 'อดุลยเดช',
 ' ',
 'เจ้